# Get Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

import psycopg2
from psycopg2 import Error

In [2]:
try:
    connection = psycopg2.connect(user="YOUR NETID",
                                  password="YOUR PASSWORD",
                                  host="pg.analytics.northwestern.edu",
                                  port="5432",
                                  database="everything2023")

    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [3]:
df = pd.read_sql("SELECT * FROM group_5.final_data;", connection);
df

,SKU,STORE,COST,RETAIL,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP
0,3,202,123.36,440.0,2005-01-18,P,1,440.0,30.00,30.00,TAMPA,FL,33612
1,3,202,123.36,440.0,2005-01-29,R,1,440.0,30.00,30.00,TAMPA,FL,33612
2,3,303,0.00,0.0,2004-08-18,P,1,0.0,12.00,12.00,ST. ANN,MO,63074
3,3,709,0.00,0.0,2005-08-14,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308
4,3,802,123.36,440.0,2005-08-09,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,2005-04-23,P,1,39.0,39.00,39.00,HOUSTON,TX,77056
120916892,9999997,7907,15.00,19.5,2005-05-03,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916893,9999997,7907,15.00,19.5,2005-05-04,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916894,9999997,7907,15.00,19.5,2005-05-04,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546


In [4]:
if (connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed") 

PostgreSQL connection is closed


## Impute missing values

In [5]:
# Replace 0s with NaN in the 'COST' column
df['COST'].replace(0, np.nan, inplace=True)

# Calculate average cost for each SKU
sku_avg_cost = df.groupby('SKU')['COST'].transform('mean')

sku_avg_cost

0            123.36
1            123.36
2            123.36
3            123.36
4            123.36
              ...  
120916891     15.00
120916892     15.00
120916893     15.00
120916894     15.00
120916895     15.00
Name: COST, Length: 120916896, dtype: float64

In [6]:
# Impute missing values in 'COST' with SKU average cost
df['COST'].fillna(sku_avg_cost, inplace=True)
df

,SKU,STORE,COST,RETAIL,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP
0,3,202,123.36,440.0,2005-01-18,P,1,440.0,30.00,30.00,TAMPA,FL,33612
1,3,202,123.36,440.0,2005-01-29,R,1,440.0,30.00,30.00,TAMPA,FL,33612
2,3,303,123.36,0.0,2004-08-18,P,1,0.0,12.00,12.00,ST. ANN,MO,63074
3,3,709,123.36,0.0,2005-08-14,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308
4,3,802,123.36,440.0,2005-08-09,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,2005-04-23,P,1,39.0,39.00,39.00,HOUSTON,TX,77056
120916892,9999997,7907,15.00,19.5,2005-05-03,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916893,9999997,7907,15.00,19.5,2005-05-04,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916894,9999997,7907,15.00,19.5,2005-05-04,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546


In [7]:
# There is no other way we can use to fix the missing value issue in COST column
df.dropna(subset=['COST'], inplace=True)
df

,SKU,STORE,COST,RETAIL,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP
0,3,202,123.36,440.0,2005-01-18,P,1,440.0,30.00,30.00,TAMPA,FL,33612
1,3,202,123.36,440.0,2005-01-29,R,1,440.0,30.00,30.00,TAMPA,FL,33612
2,3,303,123.36,0.0,2004-08-18,P,1,0.0,12.00,12.00,ST. ANN,MO,63074
3,3,709,123.36,0.0,2005-08-14,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308
4,3,802,123.36,440.0,2005-08-09,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,2005-04-23,P,1,39.0,39.00,39.00,HOUSTON,TX,77056
120916892,9999997,7907,15.00,19.5,2005-05-03,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916893,9999997,7907,15.00,19.5,2005-05-04,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916894,9999997,7907,15.00,19.5,2005-05-04,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546


In [8]:
# Convert SALEDATE to datetime
df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
df['month_year'] = pd.to_datetime(df['SALEDATE'].dt.to_period('M').dt.strftime('%Y-%m'))
df.drop('SALEDATE', axis=1, inplace=True)

In [9]:
df

,SKU,STORE,COST,RETAIL,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP,month_year
0,3,202,123.36,440.0,P,1,440.0,30.00,30.00,TAMPA,FL,33612,2005-01-01
1,3,202,123.36,440.0,R,1,440.0,30.00,30.00,TAMPA,FL,33612,2005-01-01
2,3,303,123.36,0.0,P,1,0.0,12.00,12.00,ST. ANN,MO,63074,2004-08-01
3,3,709,123.36,0.0,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308,2005-08-01
4,3,802,123.36,440.0,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761,2005-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,P,1,39.0,39.00,39.00,HOUSTON,TX,77056,2005-04-01
120916892,9999997,7907,15.00,19.5,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546,2005-05-01
120916893,9999997,7907,15.00,19.5,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546,2005-05-01
120916894,9999997,7907,15.00,19.5,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546,2005-05-01


In [10]:
filtered_df = df[df['STYPE'] == 'P']
new_df = filtered_df.groupby(['STORE', 'month_year']).agg({
    'COST': 'sum',
    'ORGPRICE': 'mean',
    'SPRICE': 'mean',
    'AMT': 'sum',
    'QUANTITY': 'sum',
    'CITY': 'first'
}).reset_index()
new_df

,STORE,month_year,COST,ORGPRICE,SPRICE,AMT,QUANTITY,CITY
0,102,2004-08-01,397048.153125,41.587266,28.507850,633330.40,22216,TAMPA
1,102,2004-09-01,449764.833499,42.377500,23.496931,596751.56,25397,TAMPA
2,102,2004-10-01,484191.378635,40.596916,30.122991,816995.77,27122,TAMPA
3,102,2004-11-01,465410.113387,41.804010,33.310324,843150.92,25312,TAMPA
4,102,2004-12-01,873969.452527,41.832542,31.417166,1505699.09,47926,TAMPA
...,...,...,...,...,...,...,...,...
4218,9909,2005-04-01,150799.454641,31.020608,23.042844,254577.34,11048,CHEYENNE
4219,9909,2005-05-01,144000.158786,33.298497,24.336645,248915.20,10228,CHEYENNE
4220,9909,2005-06-01,136023.043130,32.300830,23.481823,233691.10,9952,CHEYENNE
4221,9909,2005-07-01,180687.260000,30.176667,18.414846,268875.17,14601,CHEYENNE


In [11]:
new_df.isnull().sum()

STORE         0
month_year    0
COST          0
ORGPRICE      0
SPRICE        0
AMT           0
QUANTITY      0
CITY          0
dtype: int64

# New feature 1: Average Cost of Goods Sold (COGS) per item sold

This feature represents the average cost incurred to produce or purchase the goods that were sold within a given period. It's calculated by dividing the total Cost of Goods Sold by the number of items sold.

The interpretation of this metric can provide insights into the efficiency of the business operations and profitability. Specifically, a **lower** average COGS per item sold indicates that you are generating sales while keeping the cost of producing those goods relatively low. 

In [12]:
new_df['AvgCOGS_peritem'] = new_df['COST'] / new_df['QUANTITY']
new_df

,STORE,month_year,COST,ORGPRICE,SPRICE,AMT,QUANTITY,CITY,AvgCOGS_peritem
0,102,2004-08-01,397048.153125,41.587266,28.507850,633330.40,22216,TAMPA,17.872171
1,102,2004-09-01,449764.833499,42.377500,23.496931,596751.56,25397,TAMPA,17.709369
2,102,2004-10-01,484191.378635,40.596916,30.122991,816995.77,27122,TAMPA,17.852348
3,102,2004-11-01,465410.113387,41.804010,33.310324,843150.92,25312,TAMPA,18.386936
4,102,2004-12-01,873969.452527,41.832542,31.417166,1505699.09,47926,TAMPA,18.235810
...,...,...,...,...,...,...,...,...,...
4218,9909,2005-04-01,150799.454641,31.020608,23.042844,254577.34,11048,CHEYENNE,13.649480
4219,9909,2005-05-01,144000.158786,33.298497,24.336645,248915.20,10228,CHEYENNE,14.079014
4220,9909,2005-06-01,136023.043130,32.300830,23.481823,233691.10,9952,CHEYENNE,13.667910
4221,9909,2005-07-01,180687.260000,30.176667,18.414846,268875.17,14601,CHEYENNE,12.374992


In [13]:
new_df.min()

STORE                               102
month_year          2004-08-01 00:00:00
COST                                8.1
ORGPRICE                            0.0
SPRICE                         2.225903
AMT                                2.36
QUANTITY                              1
CITY               ABILENE             
AvgCOGS_peritem                     5.6
dtype: object

# New feature 2: Month-over-Month (MoM) Growth

In [14]:
# Gross Profit for each store
new_df['GrossProfit'] = new_df['AMT'] - new_df['COST']
new_df

,STORE,month_year,COST,ORGPRICE,SPRICE,AMT,QUANTITY,CITY,AvgCOGS_peritem,GrossProfit
0,102,2004-08-01,397048.153125,41.587266,28.507850,633330.40,22216,TAMPA,17.872171,236282.246875
1,102,2004-09-01,449764.833499,42.377500,23.496931,596751.56,25397,TAMPA,17.709369,146986.726501
2,102,2004-10-01,484191.378635,40.596916,30.122991,816995.77,27122,TAMPA,17.852348,332804.391365
3,102,2004-11-01,465410.113387,41.804010,33.310324,843150.92,25312,TAMPA,18.386936,377740.806613
4,102,2004-12-01,873969.452527,41.832542,31.417166,1505699.09,47926,TAMPA,18.235810,631729.637473
...,...,...,...,...,...,...,...,...,...,...
4218,9909,2005-04-01,150799.454641,31.020608,23.042844,254577.34,11048,CHEYENNE,13.649480,103777.885359
4219,9909,2005-05-01,144000.158786,33.298497,24.336645,248915.20,10228,CHEYENNE,14.079014,104915.041214
4220,9909,2005-06-01,136023.043130,32.300830,23.481823,233691.10,9952,CHEYENNE,13.667910,97668.056870
4221,9909,2005-07-01,180687.260000,30.176667,18.414846,268875.17,14601,CHEYENNE,12.374992,88187.910000


In [15]:
new_df['MoMGrowth'] = new_df.groupby('STORE')['GrossProfit'].pct_change()

In [16]:
average_growth_per_store = new_df.groupby('STORE')['MoMGrowth'].transform('median') # too many outliers if got mean
new_df['MoMGrowth'].fillna(average_growth_per_store, inplace=True)
new_df

,STORE,month_year,COST,ORGPRICE,SPRICE,AMT,QUANTITY,CITY,AvgCOGS_peritem,GrossProfit,MoMGrowth
0,102,2004-08-01,397048.153125,41.587266,28.507850,633330.40,22216,TAMPA,17.872171,236282.246875,-0.019383
1,102,2004-09-01,449764.833499,42.377500,23.496931,596751.56,25397,TAMPA,17.709369,146986.726501,-0.377919
2,102,2004-10-01,484191.378635,40.596916,30.122991,816995.77,27122,TAMPA,17.852348,332804.391365,1.264180
3,102,2004-11-01,465410.113387,41.804010,33.310324,843150.92,25312,TAMPA,18.386936,377740.806613,0.135024
4,102,2004-12-01,873969.452527,41.832542,31.417166,1505699.09,47926,TAMPA,18.235810,631729.637473,0.672389
...,...,...,...,...,...,...,...,...,...,...,...
4218,9909,2005-04-01,150799.454641,31.020608,23.042844,254577.34,11048,CHEYENNE,13.649480,103777.885359,0.386736
4219,9909,2005-05-01,144000.158786,33.298497,24.336645,248915.20,10228,CHEYENNE,14.079014,104915.041214,0.010958
4220,9909,2005-06-01,136023.043130,32.300830,23.481823,233691.10,9952,CHEYENNE,13.667910,97668.056870,-0.069075
4221,9909,2005-07-01,180687.260000,30.176667,18.414846,268875.17,14601,CHEYENNE,12.374992,88187.910000,-0.097065


# New Feature 3: ReturnPercentage

In [17]:
df_rperc = df.groupby(['STORE', 'month_year'])['STYPE'].apply(lambda x: (x == 'R').sum() / len(x) * 100).reset_index(name='ReturnPercentage')
new_df['ReturnPercentage'] = df_rperc['ReturnPercentage'].tolist()
new_df

,STORE,month_year,COST,ORGPRICE,SPRICE,AMT,QUANTITY,CITY,AvgCOGS_peritem,GrossProfit,MoMGrowth,ReturnPercentage
0,102,2004-08-01,397048.153125,41.587266,28.507850,633330.40,22216,TAMPA,17.872171,236282.246875,-0.019383,6.486509
1,102,2004-09-01,449764.833499,42.377500,23.496931,596751.56,25397,TAMPA,17.709369,146986.726501,-0.377919,6.446384
2,102,2004-10-01,484191.378635,40.596916,30.122991,816995.77,27122,TAMPA,17.852348,332804.391365,1.264180,7.227638
3,102,2004-11-01,465410.113387,41.804010,33.310324,843150.92,25312,TAMPA,18.386936,377740.806613,0.135024,6.732009
4,102,2004-12-01,873969.452527,41.832542,31.417166,1505699.09,47926,TAMPA,18.235810,631729.637473,0.672389,7.712157
...,...,...,...,...,...,...,...,...,...,...,...,...
4218,9909,2005-04-01,150799.454641,31.020608,23.042844,254577.34,11048,CHEYENNE,13.649480,103777.885359,0.386736,6.086365
4219,9909,2005-05-01,144000.158786,33.298497,24.336645,248915.20,10228,CHEYENNE,14.079014,104915.041214,0.010958,7.186933
4220,9909,2005-06-01,136023.043130,32.300830,23.481823,233691.10,9952,CHEYENNE,13.667910,97668.056870,-0.069075,6.369367
4221,9909,2005-07-01,180687.260000,30.176667,18.414846,268875.17,14601,CHEYENNE,12.374992,88187.910000,-0.097065,5.452308


In [18]:
final_df = new_df[['STORE', 'month_year', 'AvgCOGS_peritem', 'MoMGrowth', 'ReturnPercentage', 'CITY']]
final_df

,STORE,month_year,AvgCOGS_peritem,MoMGrowth,ReturnPercentage,CITY
0,102,2004-08-01,17.872171,-0.019383,6.486509,TAMPA
1,102,2004-09-01,17.709369,-0.377919,6.446384,TAMPA
2,102,2004-10-01,17.852348,1.264180,7.227638,TAMPA
3,102,2004-11-01,18.386936,0.135024,6.732009,TAMPA
4,102,2004-12-01,18.235810,0.672389,7.712157,TAMPA
...,...,...,...,...,...,...
4218,9909,2005-04-01,13.649480,0.386736,6.086365,CHEYENNE
4219,9909,2005-05-01,14.079014,0.010958,7.186933,CHEYENNE
4220,9909,2005-06-01,13.667910,-0.069075,6.369367,CHEYENNE
4221,9909,2005-07-01,12.374992,-0.097065,5.452308,CHEYENNE


# Store Selection

In [19]:
store_df = final_df.groupby('STORE').agg({
    'AvgCOGS_peritem': 'mean',
    'MoMGrowth': 'mean',
    'ReturnPercentage': 'mean',
    'CITY': 'first'
}).reset_index()

store_df['success'] = ((store_df['AvgCOGS_peritem'] < store_df['AvgCOGS_peritem'].quantile(0.50)) &
                       (store_df['MoMGrowth'] > store_df['MoMGrowth'].quantile(0.50)) &
                       (store_df['ReturnPercentage'] < store_df['ReturnPercentage'].quantile(0.50))).astype(int)

In [20]:
store_df

,STORE,AvgCOGS_peritem,MoMGrowth,ReturnPercentage,CITY,success
0,102,18.656348,0.102883,7.297510,TAMPA,0
1,103,17.761179,-1.857504,8.766210,ST LOUIS,0
2,107,16.975893,0.350877,8.382016,HURST,0
3,202,15.533738,0.190422,8.271629,TAMPA,0
4,203,16.687797,-0.614171,10.119624,CHESTERFIELD,0
...,...,...,...,...,...,...
326,9709,13.221052,-1.186029,6.835200,GREELEY,0
327,9804,14.891369,0.068109,7.260616,LAWTON,0
328,9806,17.679472,0.040775,4.250364,MABELVALE,0
329,9906,5.600000,NaN,0.000000,LITTLE ROCK,0


In [21]:
store_df.dropna(subset=['MoMGrowth'], inplace=True)

In [22]:
store_df.nsmallest(3, 'MoMGrowth')

,STORE,AvgCOGS_peritem,MoMGrowth,ReturnPercentage,CITY,success
18,503,14.155671,-668.679353,10.220279,ST. LOUIS,0
121,3209,15.247958,-102.705764,8.559571,SANDY,0
312,9209,14.029311,-6.488858,7.035581,PUEBLO,0


In [23]:
# We decide to dismiss the STORE 503 and 3209 due to their outliers features in terms of MoM Growth, and we will not include ST.LOUIS and SANDY as the dataset for prediction too.
store_df.drop([18, 121], axis=0, inplace=True)
store_df

,STORE,AvgCOGS_peritem,MoMGrowth,ReturnPercentage,CITY,success
0,102,18.656348,0.102883,7.297510,TAMPA,0
1,103,17.761179,-1.857504,8.766210,ST LOUIS,0
2,107,16.975893,0.350877,8.382016,HURST,0
3,202,15.533738,0.190422,8.271629,TAMPA,0
4,203,16.687797,-0.614171,10.119624,CHESTERFIELD,0
...,...,...,...,...,...,...
325,9704,13.149763,0.078428,6.546315,SHAWNEE,0
326,9709,13.221052,-1.186029,6.835200,GREELEY,0
327,9804,14.891369,0.068109,7.260616,LAWTON,0
328,9806,17.679472,0.040775,4.250364,MABELVALE,0


In [24]:
store_df['success'].value_counts()

success
0    277
1     50
Name: count, dtype: int64